# 第6回講義 演習

## 課題1. Denoising Autoencoderの実装. また, MNISTを用いて次のことを確認
- reconstruction errorが小さくなっている（学習が進んでいる）
- 重みの可視化（特徴の可視化）

In [2]:
%matplotlib inline

from __future__ import division
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from theano.tensor.shared_randomstreams import RandomStreams
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
import theano
import theano.tensor as T

rng = np.random.RandomState(1234)
theano_rng = RandomStreams(rng.randint(1234))

### 1. MNISTデータセットの読み込み

In [3]:
mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data.astype('float32'), mnist.target.astype('int32'))

mnist_X = mnist_X / 255

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2)

In [4]:
train_y = np.eye(10)[train_y].astype('int32')
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

### 2. Autoencoderをクラスとして定義

In [5]:
class Autoencoder:
    #- Constructor
    def __init__(self, visible_dim, hidden_dim, function):
        self.visible_dim = visible_dim
        self.hidden_dim = hidden_dim
        self.function = function
        self.W = theano.shared(rng.uniform(low=-0.08, high=0.08, size=(visible_dim, hidden_dim)).astype('float32'), name='W')
        self.a = theano.shared(np.zeros(visible_dim).astype('float32'), name='a')
        self.b = theano.shared(np.zeros(hidden_dim).astype('float32'), name='b')
        self.params = [self.W, self.a, self.b]
    
    #- Encoder
    def encode(self, x):
        u = T.dot(x, self.W) + self.b# WRITE ME (HINT: use self.W and self.b)
        y = self.function(u)
        return y
    
    #- Decoder (Tied Weight)
    def decode(self, x):
        u = T.dot(x, self.W.T) + self.a # WRITE ME (HINT: use self.W and self.a)
        y = self.function(u)
        return y
    
    #- Forward Propagation
    def f_prop(self, x):
        y = self.encode(x)# WRITE ME
        reconst_x = self.decode(y)# WRITE ME
        return reconst_x
    
    #- Reconstruction Error
    def reconst_error(self, x, noise):
        tilde_x = x * noise# WRITE ME (HINT: masking noise)
        reconst_x = self.f_prop(tilde_x)
        error = T.mean(T.sum(T.nnet.binary_crossentropy(reconst_x, x), axis=1))
        return error, reconst_x

### 3. 確率的勾配法 (Stochastic Gradient Descent)

In [11]:
 def sgd(params, g_params, eps=np.float32(1.0)):
    updates = OrderedDict()
    for param, g_param in zip(params, g_params):
        updates[param] = param - eps*g_param
    return updates

### 4. モデルの構築および学習

#### 4.1 Corruption level=0の場合

In [ ]:
model = Autoencoder(train_X.shape[1], 500, T.nnet.sigmoid)

x = T.fmatrix('x')
noise = T.fmatrix('noise')

cost, reconst_x = model.reconst_error(x, noise)
params = model.params
g_params = T.grad(cost=cost, wrt=params)
updates = sgd(params, g_params)

train = theano.function(inputs=[x, noise], outputs=[cost, reconst_x], updates=updates, allow_input_downcast=True, name='train')

corruption_level = np.float32(0.0)
batch_size = 100
n_batches = train_X.shape[0] // batch_size

#- Epoch
for epoch in xrange(100):
    train_X = shuffle(train_X)
    err_all = []
    for i in xrange(0, n_batches):
        start = i * batch_size
        end = start + batch_size
        
        noise = rng.binomial(size=train_X[start:end].shape, n=1, p=1-corruption_level)
        err, reconst_x = train(train_X[start:end], noise)
        err_all.append(err)
    if epoch % 10 == 0:
        print 'Epoch:%d, Error:%lf' % (epoch, np.mean(err))

weight_0 = model.W.get_value(borrow=True).T

#### 4.2 Corruption level=0.5の場合

In [ ]:
model = Autoencoder(train_X.shape[1], 500, T.nnet.sigmoid)

x = T.fmatrix('x')
noise = T.fmatrix('noise')

cost, reconst_x = model.reconst_error(x, noise)
params = model.params
g_params = T.grad(cost=cost, wrt=params)
updates = sgd(params, g_params)

train = theano.function(inputs=[x, noise], outputs=[cost, reconst_x], updates=updates, allow_input_downcast=True, name='train')

corruption_level = np.float32(0.5)
batch_size = 100
n_batches = train_X.shape[0] // batch_size

#- Epoch
for epoch in xrange(10):
    train_X = shuffle(train_X)
    err_all = []
    for i in xrange(0, n_batches):
        start = i * batch_size
        end = start + batch_size
        
        noise = rng.binomial(size=train_X[start:end].shape, n=1, p=1-corruption_level)
        err, reconst_x = train(train_X[start:end], noise)
        err_all.append(err)
    print 'Epoch:%d, Error:%lf' % (epoch, np.mean(err))

weight_1 = model.W.get_value(borrow=True).T

### 5. 重みの可視化
- corruption_levelの違いによる重みの違いを観測

#### 5.1 Corruption level=0の場合

In [ ]:
fig = plt.figure(figsize=(10, 10))
for i in xrange(100):
    ax = fig.add_subplot(10, 10, i + 1, xticks=[], yticks=[])
    ax.imshow(weight_0[i].reshape((28, 28)), cmap='gray')

#### 5.2 Corruption level=0.5の場合

In [ ]:
fig = plt.figure(figsize=(10, 10))
for i in xrange(100):
    ax = fig.add_subplot(10, 10, i + 1, xticks=[], yticks=[])
    ax.imshow(weight_1[i].reshape((28, 28)), cmap='gray')

## 課題2. Stacked Denoising Autoencoder (SdA) の実装

### 1. SdA用のAutoencoderクラス

In [14]:
class Autoencoder:
    #- Constructor
    def __init__(self, visible_dim, hidden_dim, W, function):
        self.visible_dim = visible_dim
        self.hidden_dim  = hidden_dim
        self.function    = function
        self.W           = W
        self.a = theano.shared(np.zeros(visible_dim).astype('float32'), name='a')
        self.b = theano.shared(np.zeros(hidden_dim).astype('float32'), name='b')
        self.params = [self.W, self.a, self.b]

    #- Encoder
    def encode(self, x):
        u = T.dot(x, self.W) + self.b
        y = self.function(u)
        return y

    #- Decoder
    def decode(self, x):
        u = T.dot(x, self.W.T) + self.a
        y = self.function(u)
        return y

    #- Forward Propagation
    def f_prop(self, x):
        y = self.encode(x)
        reconst_x = self.decode(y)
        return reconst_x

    #- Reconstruction Error
    def reconst_error(self, x, noise):
        tilde_x = x * noise
        reconst_x = self.f_prop(tilde_x)
        error = T.mean(T.sum(T.nnet.binary_crossentropy(reconst_x, x), axis=1))
        return error, reconst_x

### 2. SdA用のLayerクラス

In [15]:
class Layer:
    #- Constructor
    def __init__(self, in_dim, out_dim, function):
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.function = function
        self.W = theano.shared(rng.uniform(low=-0.08, high=0.08, size=(in_dim, out_dim)).astype('float32'), name='W')
        self.b = theano.shared(np.zeros(out_dim).astype('float32'), name='b')
        self.params = [self.W, self.b]

        self.set_pretraining()

    #- Forward Propagation
    def f_prop(self, x):
        self.u = T.dot(x, self.W) + self.b
        self.z = self.function(self.u)
        return self.z

    #- Set Pretraining
    def set_pretraining(self):
        ae = Autoencoder(self.in_dim, self.out_dim, self.W, self.function)

        x = T.fmatrix(name='x')
        noise = T.fmatrix(name='noise')

        cost, reconst_x = ae.reconst_error(x, noise)
        params = ae.params
        g_params = T.grad(cost=cost, wrt=params)
        updates = sgd(params, g_params)

        self.pretraining = theano.function(inputs=[x, noise], outputs=[cost, reconst_x], updates=updates, allow_input_downcast=True, name='pretraining')
        hidden = ae.encode(x)
        self.encode_function = theano.function(inputs=[x], outputs=hidden, allow_input_downcast=True, name='encode_function')

### 3. ネットワークの定義

In [20]:
layers = [Layer(train_X.shape[1], 500,T.nnet.sigmoid),
          Layer(500, 400, T.nnet.sigmoid),
          Layer(400,10,T.nnet.softmax)]

### 4. 事前学習 (Pre-training)

In [21]:
X = train_X
for l, layer in enumerate(layers[:-1]):
    corruption_level = np.float32(0.3)
    batch_size = 100
    n_batches = X.shape[0] // batch_size

    for epoch in xrange(501):
        X = shuffle(X)
        err_all = []
        for i in xrange(0, n_batches):
            start = i*batch_size
            end = start + batch_size

            noise = rng.binomial(size=X[start:end].shape, n=1, p=1-corruption_level)
            err, reconst_x = layer.pretraining(X[start:end], noise)
            err_all.append(err)
        if epoch % 20 == 0:
            print 'Pretraining:: layer:%d, Epoch:%d, Error:%lf' % (l, epoch, np.mean(err))
    X = layer.encode_function(X)

Pretraining:: layer:0, Epoch:0, Error:88.959541
Pretraining:: layer:0, Epoch:20, Error:69.456940
Pretraining:: layer:0, Epoch:40, Error:70.779068
Pretraining:: layer:0, Epoch:60, Error:68.347313
Pretraining:: layer:0, Epoch:80, Error:63.713348
Pretraining:: layer:0, Epoch:100, Error:69.128242
Pretraining:: layer:0, Epoch:120, Error:66.695427
Pretraining:: layer:0, Epoch:140, Error:65.421661
Pretraining:: layer:0, Epoch:160, Error:65.632858
Pretraining:: layer:0, Epoch:180, Error:63.464287
Pretraining:: layer:0, Epoch:200, Error:66.786118
Pretraining:: layer:0, Epoch:220, Error:62.673603
Pretraining:: layer:0, Epoch:240, Error:63.581261
Pretraining:: layer:0, Epoch:260, Error:66.324409
Pretraining:: layer:0, Epoch:280, Error:66.932785
Pretraining:: layer:0, Epoch:300, Error:64.789726
Pretraining:: layer:0, Epoch:320, Error:63.827599
Pretraining:: layer:0, Epoch:340, Error:65.669022
Pretraining:: layer:0, Epoch:360, Error:63.904781
Pretraining:: layer:0, Epoch:380, Error:65.705002
Pretra

### 5. train関数, valid関数, test関数

In [22]:
x = T.fmatrix(name='x')
t = T.imatrix(name='t')

params = []
for i, layer in enumerate(layers):
    params += layer.params
    if i == 0:
        layer_out = layer.f_prop(x)
    else:
        layer_out = layer.f_prop(layer_out)

y = layers[-1].z
cost = T.mean(T.nnet.categorical_crossentropy(y, t))

g_params = T.grad(cost=cost, wrt=params)
updates = sgd(params, g_params)

train = theano.function(inputs=[x, t], outputs=cost, updates=updates, allow_input_downcast=True, name='train')
valid = theano.function(inputs=[x, t], outputs=[cost, T.argmax(y, axis=1)], allow_input_downcast=True, name='valid')
test  = theano.function([x], T.argmax(y, axis=1), name='test')

### 6. 学習 (Fine-tuning)

In [23]:
batch_size = 100
n_batches = train_X.shape[0]//batch_size

for epoch in xrange(1001):
    train_X, train_y = shuffle(train_X, train_y)
    for i in xrange(n_batches):
        start = i*batch_size
        end = start + batch_size
        train(train_X[start:end], train_y[start:end])
    if epoch % 20 == 0:
        valid_cost, pred_y = valid(valid_X, valid_y)
        print 'EPOCH:: %i, Validation cost: %.3f, Validation F1: %.3f' % (epoch + 1, valid_cost, f1_score(np.argmax(valid_y, axis=1).astype('int32'), pred_y, average='macro'))

EPOCH:: 1, Validation cost: 0.262, Validation F1: 0.922
EPOCH:: 21, Validation cost: 0.105, Validation F1: 0.972
EPOCH:: 41, Validation cost: 0.106, Validation F1: 0.975
EPOCH:: 61, Validation cost: 0.109, Validation F1: 0.976
EPOCH:: 81, Validation cost: 0.111, Validation F1: 0.976
EPOCH:: 101, Validation cost: 0.113, Validation F1: 0.976
EPOCH:: 121, Validation cost: 0.115, Validation F1: 0.976
EPOCH:: 141, Validation cost: 0.116, Validation F1: 0.976
EPOCH:: 161, Validation cost: 0.118, Validation F1: 0.977
EPOCH:: 181, Validation cost: 0.119, Validation F1: 0.977
EPOCH:: 201, Validation cost: 0.120, Validation F1: 0.977
EPOCH:: 221, Validation cost: 0.120, Validation F1: 0.977
EPOCH:: 241, Validation cost: 0.121, Validation F1: 0.977
EPOCH:: 261, Validation cost: 0.122, Validation F1: 0.977
EPOCH:: 281, Validation cost: 0.123, Validation F1: 0.977
EPOCH:: 301, Validation cost: 0.123, Validation F1: 0.977
EPOCH:: 321, Validation cost: 0.124, Validation F1: 0.977
EPOCH:: 341, Validat